<a href="https://colab.research.google.com/github/carlibeisel/mason_water_budget/blob/main/reach_compile_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Compile data for Mason Water Budget**

By: Carli Beisel

Created: August 2024

**Variables:**

*qin* = streamflow at upstream gauge on reach

*qout* = streamflow at downstream gauge on reach

*sr* = surface runoff from agricultural irrigation

*gw* = groundwater

*pet* = potential evapotranspiration

In [1]:
## --------------- ##
## IMPORT PACKAGES ##
## --------------- ##

import pandas as pd
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
## --------------- ##
##  LOAD DATASETS  ##
## --------------- ##

#streamflow
drainflow = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/drainflow_out/final_monthly_flow.csv')
drainflow['Month'] = pd.to_datetime(drainflow['Month'], format='%Y-%m' )
drainflow['Qin (cfs)'] = drainflow['Qin']
drainflow['Qout (cfs)'] = drainflow['Qout']

# Precipitation
precip = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/precip_out/reach_precip_monthly.csv')

# Calculate days in each month dynamically
precip['Month'] = pd.to_datetime(precip['Month'], format='%Y-%m')
precip['Days in Month'] = precip['Month'].dt.days_in_month

# Convert from AF to cfs using dynamic conversion
precip['Precip (cfs)'] = (precip['Total Monthly Precip (AF)'] * 43560) / (precip['Days in Month'] * 86400)

# Evaporation (PET)
pet = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/pet_out/reach_pet.csv')
pet['Month'] = pd.to_datetime(pet['Month'], format='%m')
pet['Days in Month'] = pet['Month'].dt.days_in_month

# Convert from AF to cfs for PET
pet['PET (cfs)'] = (pet['Reach Evap (AF)'] * 43560) / (pet['Days in Month'] * 86400)

# Surface Runoff
surface_runoff = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/irrigation_runoff_out/reach_monthly_runoff_final.csv')

# Correct month formatting for surface runoff
month_mapping = {
    "jan": "01", "feb": "02", "mar": "03", "apr": "04",
    "may": "05", "jun": "06", "jul": "07", "aug": "08",
    "sep": "09", "oct": "10", "nov": "11", "dec": "12"
}
surface_runoff['month'] = surface_runoff['month'].str.lower().map(month_mapping)
surface_runoff['Month'] = pd.to_datetime("2024-" + surface_runoff['month'], format='%Y-%m')
surface_runoff['SR (cfs)'] = surface_runoff['flow(cfs)']

In [20]:
## ------------------------------ ##
##   Merge into one .csv file (cfs)
## ------------------------------ ##

drainflow_selected = drainflow[['Reach', 'Month', 'Qin (cfs)', 'Qout (cfs)']]
pet_selected = pet[['Reach', 'Month', 'PET (cfs)']]
precip_selected = precip[['Reach', 'Month', 'Precip (cfs)']]
surface_runoff_selected = surface_runoff[['Reach', 'Month', 'SR (cfs)']]

# Merge the DataFrames on "Reach" and "Month"
merged_df = drainflow_selected.merge(pet_selected, on=['Reach', 'Month'], how='outer')
merged_df = merged_df.merge(precip_selected, on=['Reach', 'Month'], how='outer')
merged_df = merged_df.merge(surface_runoff_selected, on=['Reach', 'Month'], how='outer')

# Display the final merged DataFrame
merged_df.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget_cfs.csv')

In [ ]:
## ------------------------------ ##
##   Merge into one .csv file (KAF)
## ------------------------------ ##

drainflow_selected = drainflow[['Reach', 'Month', 'Qin (KAF)', 'Qout (KAF)']]
pet_selected = pet[['Reach', 'Month', 'PET (KAF)']]
precip_selected = precip[['Reach', 'Month', 'Precip (KAF)']]
surface_runoff_selected = surface_runoff[['Reach', 'Month', 'SR (KAF)']]

# Merge the DataFrames on "Reach" and "Month"
merged_df = drainflow_selected.merge(pet_selected, on=['Reach', 'Month'], how='outer')
merged_df = merged_df.merge(precip_selected, on=['Reach', 'Month'], how='outer')
merged_df = merged_df.merge(surface_runoff_selected, on=['Reach', 'Month'], how='outer')

# Display the final merged DataFrame
merged_df.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget_kaf.csv')

In [21]:
# ---------------------------- #
# Calculate Water Budget
# --------------------------- #
def calculate_wb_residual(df):
    """
    Calculate the water budget residual (Qout - Qin) in acre-feet (AF) for each Gauge at each Month.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Gauge', 'Month', 'Qin', and 'Qout' columns.

    Returns:
    pd.DataFrame: DataFrame with an additional 'Flow Difference (AF)' column.
    """
    # Ensure 'Qin' and 'Qout' are numeric
    df['Qin (cfs)'] = pd.to_numeric(df['Qin (cfs)'], errors='coerce')
    df['Qout (cfs)'] = pd.to_numeric(df['Qout (cfs)'], errors='coerce')
    df['Precip (cfs)'] = pd.to_numeric(df['Precip (cfs)'], errors='coerce')
    df['PET (cfs)'] = pd.to_numeric(df['PET (cfs)'], errors='coerce')
    df['SR (cfs)'] = pd.to_numeric(df['SR (cfs)'], errors='coerce')

    # Calculate the flow difference ( Residual = output - input )
    df['Residual (cfs)'] = (df['Qout (cfs)'] + df['PET (cfs)']) - (df['Qin (cfs)'] + df['Precip (cfs)'])

    return df

final_df = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget_cfs.csv')
residual = calculate_wb_residual(final_df)
residual.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget_cfs.csv', index = False)